# COMEX Mako Level-3 Quicklook

This notebook visualises quicklook products for COMEX Mako whisk collections.
Set the `MAKO_COMEX_QUICKLOOK_MODE` environment variable to `synthetic` to
execute the notebook without external data. For real data, provide the
`MAKO_L2S_PATH`, `MAKO_BTEMP_PATH`, and `MAKO_ACE_PATH` environment variables
that point to matching `.dat` or `.hdr` files (the complementary file will be
located automatically).

In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from alchemi.io.mako import ACE_GAS_NAMES, open_mako_ace, open_mako_btemp, open_mako_l2s


In [ ]:
MODE = os.getenv("MAKO_COMEX_QUICKLOOK_MODE", "real").lower()
L2S_PATH = Path(os.getenv("MAKO_L2S_PATH", "/path/to/S_X_Y_L2S.dat"))
BT_PATH = Path(os.getenv("MAKO_BTEMP_PATH", "/path/to/S_X_Y_BTEMP.dat"))
ACE_PATH = Path(os.getenv("MAKO_ACE_PATH", "/path/to/S_X_Y_ACE.dat"))


In [ ]:
def _synthetic_coords():
    y = np.arange(32, dtype=np.int32)
    x = np.arange(48, dtype=np.int32)
    band = np.arange(128, dtype=np.int32)
    wavelengths_nm = np.linspace(7500.0, 13500.0, band.size, dtype=np.float64)
    return y, x, band, wavelengths_nm


def _synthetic_l2s():
    y, x, band, wavelengths_nm = _synthetic_coords()
    rng = np.random.default_rng(0)
    radiance = rng.normal(0.01, 0.001, size=(y.size, x.size, band.size))
    ds = xr.Dataset(coords={"y": y, "x": x, "band": band})
    ds = ds.assign_coords(wavelength_nm=("band", wavelengths_nm))
    ds["radiance"] = xr.DataArray(
        radiance,
        dims=("y", "x", "band"),
        coords={"y": y, "x": x, "band": band, "wavelength_nm": ("band", wavelengths_nm)},
        attrs={"units": "W·m⁻²·sr⁻¹·nm⁻¹"},
    )
    ds.attrs["sensor"] = "mako"
    return ds


def _synthetic_bt():
    y, x, band, wavelengths_nm = _synthetic_coords()
    base = 290.0 + 5.0 * np.sin(np.linspace(0, np.pi, band.size))
    bt = base[None, None, :] + 0.5 * np.sin(np.linspace(0, 4 * np.pi, y.size))[:, None, None]
    bt = bt + 0.5 * np.cos(np.linspace(0, 4 * np.pi, x.size))[None, :, None]
    ds = xr.Dataset(coords={"y": y, "x": x, "band": band})
    ds = ds.assign_coords(wavelength_nm=("band", wavelengths_nm))
    ds["bt"] = xr.DataArray(
        bt,
        dims=("y", "x", "band"),
        coords={"y": y, "x": x, "band": band, "wavelength_nm": ("band", wavelengths_nm)},
        attrs={"units": "K"},
    )
    ds.attrs["sensor"] = "mako"
    return ds


def _synthetic_ace():
    y, x, _, _ = _synthetic_coords()
    rng = np.random.default_rng(1)
    ace = rng.uniform(-1.0, 1.0, size=(y.size, x.size, len(ACE_GAS_NAMES)))
    gas_band = np.arange(len(ACE_GAS_NAMES), dtype=np.int32)
    coords = {
        "y": y,
        "x": x,
        "gas_band": gas_band,
        "gas_name": xr.IndexVariable("gas_band", ACE_GAS_NAMES),
    }
    ds = xr.Dataset(coords=coords)
    ds["ace"] = xr.DataArray(
        ace,
        dims=("y", "x", "gas_band"),
        coords={**coords, "gas_name": ("gas_band", ACE_GAS_NAMES)},
        attrs={"units": "dimensionless"},
    )
    ds.attrs["sensor"] = "mako"
    return ds


In [ ]:
def load_quicklook_datasets():
    if MODE == "synthetic":
        return _synthetic_l2s(), _synthetic_bt(), _synthetic_ace()
    l2s = open_mako_l2s(L2S_PATH)
    bt = open_mako_btemp(BT_PATH)
    ace = open_mako_ace(ACE_PATH)
    return l2s, bt, ace


l2s_ds, bt_ds, ace_ds = load_quicklook_datasets()


In [ ]:
bt_map = bt_ds["bt"].mean(dim="band")
if "gas_name" in ace_ds.coords:
    gas_names = np.asarray(ace_ds["gas_name"].values)
    matches = np.where(gas_names == "CH4")[0]
    methane_index = int(matches[0]) if matches.size else 2
else:
    methane_index = 2
methane_map = ace_ds["ace"].isel(gas_band=methane_index)

fig, axes = plt.subplots(1, 2, figsize=(12, 5), constrained_layout=True)

im0 = axes[0].imshow(bt_map, cmap="inferno")
axes[0].set_title("Brightness temperature (band average)")
axes[0].set_xlabel("x")
axes[0].set_ylabel("y")
fig.colorbar(im0, ax=axes[0], label="K")

im1 = axes[1].imshow(methane_map, cmap="coolwarm", vmin=-1.0, vmax=1.0)
axes[1].set_title("ACE methane score")
axes[1].set_xlabel("x")
axes[1].set_ylabel("y")
fig.colorbar(im1, ax=axes[1], label="score")

plt.show()
